In [10]:
import cv2
import glob
import pandas as pd
from typing import List
from tqdm import tqdm
from pyzbar.pyzbar import decode
from hcmus.core import pconf

In [2]:
def list_all_images(folder_path: str) -> List[str]: 
    image_files = []
    extensions = ["jpg", "png", "JPG", "PNG"]
    for ext in extensions:
        image_files.extend(glob.glob(f"{folder_path}/**/*.{ext}", recursive=True))

    return image_files

In [3]:
def scan_barcode(image_path: str):
    image = cv2.imread(image_path)
    barcodes = decode(image)
    result = [] 
    if barcodes:
        for barcode in barcodes:
            barcode_data = barcode.data.decode("utf-8")
            barcode_type = barcode.type
            result.append({
                "data": barcode_data,
                "type": barcode_type,
                "path": image_path
            })

    return result

In [4]:
image_paths = list_all_images(pconf.IMPORT_DATA_DIR)

In [5]:
all_barcodes = []
for image_path in tqdm(image_paths):
    all_barcodes.extend(scan_barcode(image_path))

 11%|█         | 170/1598 [00:48<06:04,  3.92it/s]WARNING: decoder/databar.c:1211: _zbar_decode_databar: Assertion "seg->finder >= 0" failed.
	i=29 f=-1(010) part=1
 63%|██████▎   | 1000/1598 [04:21<04:27,  2.23it/s]WARNING: decoder/databar.c:1211: _zbar_decode_databar: Assertion "seg->finder >= 0" failed.
	i=24 f=-1(000) part=0
 83%|████████▎ | 1324/1598 [05:54<01:11,  3.83it/s]WARNING: decoder/databar.c:1211: _zbar_decode_databar: Assertion "seg->finder >= 0" failed.
	i=21 f=-1(010) part=0
 93%|█████████▎| 1479/1598 [06:39<00:23,  5.11it/s]WARNING: decoder/databar.c:1211: _zbar_decode_databar: Assertion "seg->finder >= 0" failed.
	i=31 f=-1(101) part=0
100%|██████████| 1598/1598 [06:53<00:00,  3.87it/s]


In [12]:
df = pd.read_json("2.scanned-barcodes.json")
df = df.rename(columns={
    "data": "Barcode",
    "path": "Example File"
})
df.head(3)

,Barcode,type,Example File
0,8801051128178,EAN13,/Volumes/Cucumber/Projects/item-identification...
1,8935030221359,EAN13,/Volumes/Cucumber/Projects/item-identification...
2,8934868186540,EAN13,/Volumes/Cucumber/Projects/item-identification...


In [14]:
df = df[df["type"] == "EAN13"]
df = df.drop_duplicates(subset=["Barcode"], keep="first")

In [15]:
df.to_csv("2.scanned-barcodes.csv")